In [9]:
import pandas as pd
from datetime import datetime, timedelta
from dateutil import parser
import time
import requests

In [15]:
tokenApi = ['enb4iHROSDegvkiZv17FxzISZ%2BZB%2FQxkSGweAsL8sB0%3D']
url = "https://elysia.zeev.it"
tipoUrl = "/api/2/assignments"

# Periodo de selecao
today = datetime.today().strftime("%Y-%m-%d")
periodo1 = "2023-03-17"
periodo2 = "2023-03-10"

2023-03-20


In [101]:
# Se a API alterar seu campos novamente, sera necessario alterar apenas alguns filtros desta funcao
def request_Orquestra(method='GET',urlReq="https://elysia.zeev.it",typeReq="/api/2/assignments",tokens=[],filters={}, forceRequisition:bool=False) -> list:
    # Variaveis de controle
    row = 1
    i = 1
    retorno = []
    # filtros base para o payload
    print("Requisicao para {} <<".format(urlReq+typeReq))
    if not typeReq.lower().__contains__('report'):
        filters['recordsPerPage'] = 100
        seconds = 1
    else:
        if not forceRequisition: 
            print("---- >> Requisicao lenta... << ---- ")
            filters['recordsPerPage'] = 30
            seconds = 3
        else:
            print("---- >> Requisicao forçada... << ---- ")
            filters['recordsPerPage'] = 100
            seconds = 1    
    
    filters['mobileEnabledOnly'] = False
    # Faz a busca para cada token
    for tk in tokens:
        # Reseta variaveis de controle
        row = 1
        i = 1
        # Iteracao com token
        authorization= {'Authorization': "Bearer " + tk[0]}
        while row != 0:
            # Aguardo x segundos devido ao limite de requisicoes - lembrando que a limite de requisicoes por segundos, minutos, horas, dias e meses
            time.sleep(seconds) 
            
            # Incremento de pagina
            filters['pageNumber'] = i
            
            # Requisicao
            print(filters)
            getReq = requests_API_Orquestra(metodo=method, 
                                            urlAcesso=urlReq, 
                                            tipoAcesso=typeReq, 
                                            head=authorization, 
                                            payload=filters)
        
            # Resultado JSON da requisicao
            result = getReq.json()

            # Checa se esta vazio o resultado, o que significa que nao tem mais paginas
            if (len(result) == 0) or (getReq.status_code != 200):
                    print(">> " + str(getReq.status_code))
                    break
            else:
                    # coloca cada retorno na lista
                    retorno.append(result)
                    i+=1
    # Cada usuario do token fica em uma sublista da lista de retorno, por isto a necessidade de 'explodir' ela dentro de outra lista.
    retornoLista = [item for sublista in retorno for item in sublista]
    return retornoLista 

def requests_API_Orquestra(metodo='GET',urlAcesso="https://elysia.zeev.it",tipoAcesso="/api/2/assignments",head={},payload={}):
    '''
        Funcao
        ------
            Funcao que retorna a requisicao desejada a partir do url de acesso e link de tipo de acesso.
            !Sendo permitido apenas metodos POST e GET!

        Parameters
        ----------

            metodo: str (OPCIONAL)
                Metodo a ser utilizado na requisicao, POST ou GET.
                Se nao for especificado nenhum ele ira utilizar o metodo 'GET'.
            
            urlAcesso: str (OPCIONAL)
                Url usada na requisicao.
                Se nao for especificado nenhum ele ira utilizar o URL que ja esta no funcao.
            
            tipoAcesso: str (OPCIONAL)
                Url de acesso para o tipo de funcao na API, POST ou GET.
                Se nao for especificado nenhum ele ira utilizar o URL que ja esta no funcao.
                Ex.: **/api/2/assignments**
            
            head: str (OBRIGATORIO)
                Autorizacao para acesso da API.
                Ex.: **{'Authorization': 'token_acesso'}**
            
            payload: str (OPCIONAL)
                Parametros para acesso atraves de filtros na requisicao.
        
        Retorno
        -------
            Retorna o objeto response ou resposta de erro caso ocorra.
            Ex.: <Response [200]> 
    '''
    # verifica qual tipo de requisicao foi pedida
    if metodo == 'GET':
        req = requests.get(url=urlAcesso+tipoAcesso,headers=head,params=payload)
        return req
    elif metodo == 'POST':
        req = requests.post(url=urlAcesso+tipoAcesso,headers=head,params=payload)
        return req
    else: 
        raise NameError('Nenhum metodo valido foi passado. Passe os metodos POST ou GET.')
def instances_Report_Orquestra(userToken:str, showFinishedInstanceTasks:bool=True, showPendingInstanceTasks:bool=False, activeInstances:bool=True, forceReq:bool=False, flowId:int=0, periodo:bool=False):
    '''
        Funcao
        ------
            Lista todas instâncias de solicitações que a pessoa relacionada ao token possui permissão de consultar de acordo com filtros.
        
        Parametros
        ----------
            userToken : str, (obrigatorio)
                Token do usuario orquestra para solicitar a requisicao.
            
            Parametros de especial atencao:
            -------------------------------

            `showFinishedTasks` : boolean, (opcional)
                Define se vai requisitar instancias fechadas das solicitacoes.
            
            `showPendingInstanceTasks` : boolean, (opcional)
                Define se vai requisitar instancias abertas das solicitacoes.

            * Se nao optar por `showPendingInstanceTasks` ou `showFinishedTasks`, sera necessario colocar o parametro `forceReq` como `True`.\n
            * Se `showPendingInstanceTasks` ou `showFinishedTasks` estiverem `True` o parametro `forceReq` eh alterado para `False`, mesmo se ele for passado pela funcao.
            
            forceReq : boolean, (opcional)
                Este parametro limita a quantidade de solicitacoes por segundo.

            activeInstances : boolean, (opcional)
                Definir se a solicitacao esta `Finalizada` ou `Em andamento`.

            flowId : str, (opcional)
                Filtrar pelo id de instancia.
            
            periodo : boolean, (opcional)
                Define se o periodo vai ser utilizado
        Retorno 
        -------
            Retorna um DataFrame(Pandas).
            * Se erro, retorna -1.
    '''
    # Filtro padrao - Solicitacoes Em andamento e com instancias Finalizadas
    f = {"startDateIntervalBegin": "2000-01-01T00:00:00",
            "startDateIntervalEnd": "2030-12-31T23:59:59",
            "showFinishedInstanceTasks": showFinishedInstanceTasks, 
            "showPendingInstanceTasks": showPendingInstanceTasks,
            "formFieldNames":["numeroDoProjeto",
                              "cidade",
                              "origemDoCliente",
                              "formatoEvento",
                              "executivoDeVendas",
                              "preVendedor",
                              "nomeDoCliente",
                              "dataDoAgendamento",
                              "observacoes"
                              ],
            "active": activeInstances}

    # Filtro de ID para solicitacoes especificas
    if flowId != 0: 
        f['flowId'] = flowId
    
    # Usar periodo de pesquisa
    if periodo:
        f['startDateIntervalBegin'] = '2023-01-01' + "T00:00:00"
        f['startDateIntervalEnd'] = today + "T23:59:59"
    
    if showFinishedInstanceTasks == True | showPendingInstanceTasks == True:
        forceReq = False
        
    lista = request_Orquestra(method='GET',
                                urlReq=url,
                                typeReq='/api/2/instances/report',
                                tokens=[userToken],
                                filters=f,
                                forceRequisition=forceReq)
    df = pd.DataFrame(lista)
    return df

In [102]:
# ---- Inicio do codigo ----
# Buscar o dados do fluxo 
# Numero do fluxo: 368
instanciasInativas = instances_Report_Orquestra(tokenApi, True, True, False, flowId=368, periodo=True) # Inativas
instanciasAtivas = instances_Report_Orquestra(tokenApi, True, True, True, flowId=368, periodo=True) # Ativas

Requisicao para https://elysia.zeev.it/api/2/instances/report <<
---- >> Requisicao lenta... << ---- 
{'startDateIntervalBegin': '2023-01-01T00:00:00', 'startDateIntervalEnd': '2023-03-20T23:59:59', 'showFinishedInstanceTasks': True, 'showPendingInstanceTasks': True, 'formFieldNames': ['numeroDoProjeto', 'cidade', 'origemDoCliente', 'formatoEvento', 'executivoDeVendas', 'preVendedor', 'nomeDoCliente', 'dataDoAgendamento', 'observacoes'], 'active': False, 'flowId': 368, 'recordsPerPage': 30, 'mobileEnabledOnly': False, 'pageNumber': 1}
{'startDateIntervalBegin': '2023-01-01T00:00:00', 'startDateIntervalEnd': '2023-03-20T23:59:59', 'showFinishedInstanceTasks': True, 'showPendingInstanceTasks': True, 'formFieldNames': ['numeroDoProjeto', 'cidade', 'origemDoCliente', 'formatoEvento', 'executivoDeVendas', 'preVendedor', 'nomeDoCliente', 'dataDoAgendamento', 'observacoes'], 'active': False, 'flowId': 368, 'recordsPerPage': 30, 'mobileEnabledOnly': False, 'pageNumber': 2}
>> 200
Requisicao pa

In [103]:
frames = [instanciasAtivas, instanciasInativas]
instancias = pd.concat(frames)
instancias

,id,masterInstanceId,starterInstanceId,requestName,reportLink,confirmationCode,uid,simulation,active,flowResult,...,startDateTime,endDateTime,cancelUserId,lastFinishedTaskDateTime,leadTimeInDays,flow,service,formFields,requester,instanceTasks
0,63315,None,None,Agenda Comercial Microsoft v. 1,https://elysia.zeev.it/2.0/audit?c=eaBKhIga%2F...,NTY0J4,99ed1bed-de29-4667-89fe-99f9edbb598e,False,True,,...,2023-03-20T08:05:16,None,None,2023-03-20T08:38:56,None,"{'id': 368, 'uid': '973bfffa-9555-4ce3-b0b5-b6...",None,"[{'id': 196217764, 'name': 'nomeDoCliente', 'v...","{'id': 1884, 'name': 'Anônimo', 'email': 'serv...","[{'id': 1047194, 'startDateTime': '2023-03-20T..."
1,63027,None,None,Agenda Comercial Microsoft v. 1,https://elysia.zeev.it/2.0/audit?c=eaBKhIga%2F...,NTY0J4,613c0e08-70f0-4544-b338-540801f0a21b,False,True,,...,2023-03-15T10:07:40,None,None,2023-03-17T00:07:42,None,"{'id': 368, 'uid': '973bfffa-9555-4ce3-b0b5-b6...",None,"[{'id': 190479387, 'name': 'nomeDoCliente', 'v...","{'id': 1884, 'name': 'Anônimo', 'email': 'serv...","[{'id': 1023643, 'startDateTime': '2023-03-17T..."
2,63016,None,None,Agenda Comercial Microsoft v. 1,https://elysia.zeev.it/2.0/audit?c=eaBKhIga%2F...,NTY0J4,b4dcf895-5012-409d-be03-2ff1145fb6bd,False,True,,...,2023-03-14T15:04:23,None,None,2023-03-16T03:10:02,None,"{'id': 368, 'uid': '973bfffa-9555-4ce3-b0b5-b6...",None,"[{'id': 188978002, 'name': 'nomeDoCliente', 'v...","{'id': 1884, 'name': 'Anônimo', 'email': 'serv...","[{'id': 1017491, 'startDateTime': '2023-03-16T..."
0,63333,None,None,Agenda Comercial Microsoft v. 1,https://elysia.zeev.it/2.0/audit?c=eaBKhIga%2F...,NTY0J4,04dda695-904e-44a2-8680-f80efdfb255b,False,False,Concluído,...,2023-03-20T11:39:46,2023-03-20T11:39:48,None,2023-03-20T11:39:48,0.0,"{'id': 368, 'uid': '973bfffa-9555-4ce3-b0b5-b6...",None,"[{'id': 196446790, 'name': 'nomeDoCliente', 'v...","{'id': 1884, 'name': 'Anônimo', 'email': 'serv...","[{'id': 1048136, 'startDateTime': '2023-03-20T..."
1,63324,None,None,Agenda Comercial Microsoft v. 1,https://elysia.zeev.it/2.0/audit?c=eaBKhIga%2F...,NTY0J4,be55c433-5659-413b-935b-c73f4f759063,False,False,Concluído,...,2023-03-20T10:08:28,2023-03-20T10:08:30,None,2023-03-20T10:08:29,0.0,"{'id': 368, 'uid': '973bfffa-9555-4ce3-b0b5-b6...",None,"[{'id': 196325008, 'name': 'nomeDoCliente', 'v...","{'id': 1884, 'name': 'Anônimo', 'email': 'serv...","[{'id': 1047610, 'startDateTime': '2023-03-20T..."
2,63173,None,None,Agenda Comercial Microsoft v. 1,https://elysia.zeev.it/2.0/audit?c=eaBKhIga%2F...,NTY0J4,57d2a581-ba6c-4b10-9ec2-b091af0ce1fe,False,False,Concluído,...,2023-03-17T17:21:29,2023-03-17T17:21:31,None,2023-03-17T17:21:31,0.0,"{'id': 368, 'uid': '973bfffa-9555-4ce3-b0b5-b6...",None,"[{'id': 191774190, 'name': 'nomeDoCliente', 'v...","{'id': 1885, 'name': 'Diego Duarte Paz', 'emai...","[{'id': 1028980, 'startDateTime': '2023-03-17T..."
3,63169,None,None,Agenda Comercial Microsoft v. 1,https://elysia.zeev.it/2.0/audit?c=eaBKhIga%2F...,NTY0J4,f341b84e-7ab2-47cf-96ed-0fa34000ce9d,False,False,Concluído,...,2023-03-17T16:15:58,2023-03-17T16:15:59,None,2023-03-17T16:15:59,0.0,"{'id': 368, 'uid': '973bfffa-9555-4ce3-b0b5-b6...",None,"[{'id': 191696612, 'name': 'nomeDoCliente', 'v...","{'id': 1884, 'name': 'Anônimo', 'email': 'serv...","[{'id': 1028658, 'startDateTime': '2023-03-17T..."
4,63153,None,None,Agenda Comercial Microsoft v. 1,https://elysia.zeev.it/2.0/audit?c=eaBKhIga%2F...,NTY0J4,88915175-1dcc-42a4-9398-9253bf358c04,False,False,Concluído,...,2023-03-17T11:55:12,2023-03-17T11:55:18,None,2023-03-17T11:55:16,0.0,"{'id': 368, 'uid': '973bfffa-9555-4ce3-b0b5-b6...",None,"[{'id': 191342364, 'name': 'nomeDoCliente', 'v...","{'id': 1884, 'name': 'Anônimo', 'email': 'serv...","[{'id': 1027191, 'startDateTime': '2023-03-17T..."
5,63048,None,None,Agenda Comercial Microsoft v. 1,https://elysia.zeev.it/2.0/audit?c=eaBKhIga%2F...,NTY0J4,d8f91db1-f6cd-4c34-8690-06066bbbca4a,False,False,Concluído,...,2023-03-15T15:52:19,2023-03-15T15:52:20,Non

In [105]:
arr = instancias['formFields'].array
listDict = [dic for dic in arr]
df = pd.DataFrame(columns=['Projeto', 'Cidade', 'Origem', 'Formato', 'Executivo','Pré-Venda','Cliente','Data da Visita', 'Observações'])
valores = []
index = 0
for i in listDict:
    for dicio in i:
       valores.append(dicio['value'])
    df.loc[index] = valores
    index += 1
    valores = []
df

,Projeto,Cidade,Origem,Formato,Executivo,Pré-Venda,Cliente,Data da Visita,Observações
0,Juliana Hinterlang,12021,Inbound,Novo Hamburgo,Diego Paz,Heloisa Zanetti,Presencial,17/03/2023,
1,Rodrigo Minella Dipp,12026,Inbound,Vacaria,Tomás Zanetti,Vinícius Pizzinato,Online,17/03/2023,
2,Pedro Soster (Saf Holland),12023,Inbound,Alvorada,Tomás Zanetti,Heloisa Zanetti,Presencial,15/03/2023,
3,JBS,12033,Outbound,Montenegro,Tomás Zanetti,Maurício Weber,Online,22/03/2023,
4,HP Nutri,12032,Outbound,Camaquã,Diego Paz,Vinícius Pizzinato,Presencial,22/03/2023,
5,Transportadora Dinâmica,10725,Expansão,Cachoeirinha,Diego Paz,Vinícius Pizzinato,Presencial,20/03/2023,
6,Quero-Quero,12030,Outbound,Montenegro,Tomás Zanetti,Vinícius Pizzinato,Presencial,21/03/2023,
7,Infortel,10915,Outbound,Canoas,Tomás Zanetti,Vinícius Pizzinato,Presencial,21/03/2023,
8,TERMO ACO,10228,Outbound,Caxias do Sul,Tomás Zanetti,Maurício Weber,Presencial,22/03/2023,
9,Biopro(Tênis Clube Partenon),12027,Outbound,Porto Alegre,Tomás Zanetti,Vinícius Pizzinato,Presencial,17/03/2023,
